# Spreadsheet

    Make a spreadsheet using pinkfish.  This is useful for developing trading strategies.
    It can also be used as a tool for buy and sell signals that you then manually execute.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

from talib.abstract import *

import pinkfish as pf
import pinkfish.itable as itable

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

# Show all columns.
pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)
ts.tail()

[*********************100%***********************]  1 of 1 completed


,open,high,low,close,adj_close,volume
date,,,,,,
2022-12-23,379.65,383.06,378.03,382.91,382.91,59761100
2022-12-27,382.79,383.15,379.65,381.40,381.40,51638200
2022-12-28,381.33,383.39,376.42,376.66,376.66,70911500
2022-12-29,379.63,384.35,379.08,383.44,383.44,66970900
2022-12-30,380.64,382.58,378.43,382.43,382.43,83975100


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [5]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts.head()

,open,high,low,close,adj_close,volume
date,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,25.22,1003200
1993-02-01,43.97,44.25,43.97,44.25,25.40,480500
1993-02-02,44.22,44.38,44.12,44.34,25.45,201300
1993-02-03,44.41,44.84,44.38,44.81,25.72,529400
1993-02-04,44.97,45.09,44.47,45.00,25.83,531500


Add technical indicators

In [6]:
# Add 200 day MA.
ts['sma200'] = pf.SMA(ts, timeperiod=200)

# 200 sma regime filter.
ts['regime'] = pf.CROSSOVER(ts, timeperiod_fast=1, timeperiod_slow=200)

# X day sma.
ts['sma70'] = pf.SMA(ts, timeperiod=70)

# Add ATR.
ts['atr'] = ATR(ts, timeperiod=14)

# Add 7 day high, and 7 day low
ts['high7'] = pd.Series(ts.close).rolling(window=7).max()
ts['low7'] = pd.Series(ts.close).rolling(window=7).min()

# Add RSI, and 2-period cumulative RSI
ts['rsi2'] = RSI(ts, timeperiod=2)
ts['c2rsi2'] = pd.Series(ts.rsi2).rolling(window=2).sum()

# Add midpoint
ts['mp'] = (ts.high + ts.low) / 2

# Add 10 day SMA of midpoint
ts['sma10'] = pd.Series(ts.mp).rolling(window=10).mean()

# Add temporary rolling 10 day Standard Deviation of midpoint
ts['__sd__'] = pd.Series(ts.mp).rolling(window=10).std()

# Add standard deviation envelope or channel around midpoint
ts['upper'] = ts.sma10 + ts['__sd__']*2
ts['lower'] = ts.sma10 - ts['__sd__']*2

# Drop temporary columns.
ts.drop(columns=['__sd__'], inplace=True)

Finalize timeseries

In [7]:
ts, start = pf.finalize_timeseries(ts, start, dropna=True)
ts.tail()

,open,high,low,close,adj_close,volume,sma200,regime,sma70,atr,high7,low7,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,,,
2022-12-23,379.65,383.06,378.03,382.91,382.91,59761100,400.79,-15.00,382.66,7.17,389.63,380.02,52.62,83.82,380.54,388.44,407.01,369.87
2022-12-27,382.79,383.15,379.65,381.40,381.40,51638200,400.60,-16.00,382.60,6.91,386.23,380.02,36.81,89.43,381.40,386.96,405.14,368.78
2022-12-28,381.33,383.39,376.42,376.66,376.66,70911500,400.40,-17.00,382.43,6.92,386.23,376.66,12.76,49.57,379.91,384.48,398.04,370.91
2022-12-29,379.63,384.35,379.08,383.44,383.44,66970900,400.18,-18.00,382.42,6.97,386.23,376.66,69.60,82.35,381.71,382.56,389.71,375.41
2022-12-30,380.64,382.58,378.43,382.43,382.43,83975100,399.92,-19.00,382.49,6.83,386.23,376.66,58.28,127.88,380.50,381.45,384.84,378.07


Select a smaller time from for use with itable

In [8]:
df = ts['2022-01-01':]
df.head()

,open,high,low,close,adj_close,volume,sma200,regime,sma70,atr,high7,low7,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,,,
2022-01-03,476.30,477.85,473.85,477.71,470.08,72668200,437.64,383.00,457.75,5.44,477.71,470.60,76.40,105.50,475.85,470.69,487.74,453.65
2022-01-04,479.22,479.98,475.58,477.55,469.93,71178700,438.07,384.00,458.23,5.37,477.71,474.96,70.89,147.29,477.78,473.14,485.45,460.84
2022-01-05,477.16,477.98,468.28,468.38,460.90,104538900,438.46,385.00,458.60,5.68,477.71,468.38,7.66,78.55,473.13,474.48,482.47,466.49
2022-01-06,467.89,470.82,465.43,467.94,460.47,86858900,438.86,386.00,459.09,5.66,477.71,467.94,7.06,14.71,468.13,474.77,481.33,468.21
2022-01-07,467.95,469.20,464.65,466.09,458.65,85111600,439.24,387.00,459.54,5.58,477.71,466.09,4.24,11.30,466.93,474.42,482.26,466.59


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [9]:
pt = itable.PrettyTable(
    df, tstyle=itable.TableStyle(theme='theme1'), header_row=True, rpt_header=20)

pt.update_col_header_style(
    format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(
    format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] in ('volume', 'regime'):
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high7'][row] == pt.df['close'][row]):
        col = df.columns.get_loc('high7')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low7'][row] == pt.df['close'][row]):
        col = df.columns.get_loc('low7')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')
    if (pt.df['regime'][row] > 0 or pt.df['close'][row] > pt.df['sma70'][row]):
        col = df.columns.get_loc('regime')
        pt.update_cell_style(rows=[row], cols=[col], color='green')

In [10]:
pt

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,SMA200,REGIME,SMA70,ATR,HIGH7,LOW7,RSI2,C2RSI2,MP,SMA10,UPPER,LOWER
2022/01/03,476.30,477.85,473.85,477.71,470.08,72668200,437.64,383,457.75,5.44,477.71,470.60,76.40,105.50,475.85,470.69,487.74,453.65
2022/01/04,479.22,479.98,475.58,477.55,469.93,71178700,438.07,384,458.23,5.37,477.71,474.96,70.89,147.29,477.78,473.14,485.45,460.84
2022/01/05,477.16,477.98,468.28,468.38,460.90,104538900,438.46,385,458.60,5.68,477.71,468.38,7.66,78.55,473.13,474.48,482.47,466.49
2022/01/06,467.89,470.82,465.43,467.94,460.47,86858900,438.86,386,459.09,5.66,477.71,467.94,7.06,14.71,468.13,474.77,481.33,468.21
2022/01/07,467.95,469.20,464.65,466.09,458.65,85111600,439.24,387,459.54,5.58,477.71,466.09,4.24,11.30,466.93,474.42,482.26,466.59
2022/01/10,462.70,465.74,456.60,465.51,458.08,119362000,439.59,388,460.06,5.86,477.71,465.51,3.39,7.64,461.17,473.08,484.54,461.61
2022/01/11,465.23,469.85,462.05,469.75,462.25,74303100,439.96,389,460.57,6.00,477.71,465.51,75.38,78.77,465.95,471.93,483.74,460.11
2022/01/12,471.59,473.20,468.94,471.02,463.50,67605400,440.34,390,461.17,5.87,477.55,465.51,82.98,158.36,471.07,471.31,482.52,460.10
2022/01/13,472.19,472.88,463.44,464.53,457.11,91173100,440.68,391,461.62,6.13,471.02,464.53,19.97,102.95,468.16,470.39,480.89,459.89
2022/01/14,461.19,465.09,459.90,464.72,457.30,95890900,441.00,392,462.05,6.06,471.02,464.53,23.38,43.35,462.49,469.07,479.90,458.23
